In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append('..')

import autorobot as ar
from autorobot.constants import (
    RProjType,
)

In [4]:
rb = ar.initialize()

In [5]:
from autorobot.synonyms import synonyms
synonyms.update({'seashell': ar.RProjType.SHELL})

print(ar.app.synonyms)

{'BUILDING': <RProjType.BUILDING: 19>, 'FRAME_2D': <RProjType.FRAME_2D: 1>, 'FRAME_3D': <RProjType.FRAME_3D: 4>, 'SHELL': <RProjType.SHELL: 7>, 'TRUSS_2D': <RProjType.TRUSS_2D: 2>, 'TRUSS_3D': <RProjType.TRUSS_3D: 5>, 'PERM': <RCaseNature.PERM: 0>, 'IMPOSED': <RCaseNature.IMPOSED: 1>, 'WIND': <RCaseNature.WIND: 2>, 'SNOW': <RCaseNature.SNOW: 3>, 'ACC': <RCaseNature.ACC: 5>, 'SLS': <RCombType.SLS: 1>, 'ULS': <RCombType.ULS: 0>, 'LINEAR': <RAnalysisType.LINEAR: 1>, 'NON_LIN': <RAnalysisType.NON_LIN: 2>, 'COMB_LINEAR': <RAnalysisType.COMB_LINEAR: 0>, 'COMB_NON_LIN': <RAnalysisType.COMB_NON_LIN: -1>, <RCaseNature.PERM: 0>: 0, <RCaseNature.IMPOSED: 1>: 1, <RCaseNature.WIND: 2>: 2, <RCaseNature.SNOW: 3>: 3, <RCaseNature.ACC: 5>: 5, <RProjType.BUILDING: 19>: 19, <RProjType.FRAME_3D: 4>: 4, <RProjType.SHELL: 7>: 7, <RAnalysisType.COMB_NON_LIN: -1>: -1, 'seashell': <RProjType.SHELL: 7>}


In [6]:
rb.new('seashell')

In [7]:
print(list(n for n in dir(ar.RobotOM) if len(n) == max(len(s) for s in dir(ar.RobotOM))))

['_IRConcrSlabRequiredReinfEngineEvents_ProgressNotifyEventHandler']


In [8]:
ar.create_section('50x50 sq', 50., 50., shape='rect')

In [9]:
sec = ar.RobotOM.IRobotLabel(rb.structure.Labels.Get(ar.RobotOM.IRobotLabelType.I_LT_BAR_SECTION, '50x50 sq'))
data = ar.RobotOM.IRobotBarSectionData(sec.Data)

In [10]:
print(dir(data))
print(data.GetValue(ar.RobotOM.IRobotBarSectionDataValue.I_BSDV_WY))
print(dir(ar.RobotOM.IRobotBarSectionDataValue))

for k in dir(ar.RobotOM.IRobotBarSectionDataValue):
    if k.startswith('I_BSDV'):
        print(f'{k}: {data.GetValue(getattr(ar.RobotOM.IRobotBarSectionDataValue, k))}')

['AliasCount', 'CalcNonstdGeometry', 'Concrete', 'CreateNonstd', 'DrawSymbol', 'ElasticParams', 'FindAlias', 'FindNonstd', 'GetAlias', 'GetAliasEx', 'GetNonstd', 'GetValue', 'IsConcrete', 'IsJoist', 'IsSpecial', 'LoadFromDBase', 'LoadFromDBase2', 'MaterialName', 'Members', 'Name', 'NonstdCount', 'RemoveNonstd', 'SetValue', 'ShapeType', 'Special', 'Type', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'get_AliasCount', 'get_Concrete', 'get_ElasticParams', 'get_IsConcrete', 'get_IsJoist', 'get_IsSpecial', 'get_MaterialName', 'get_Members', 'get_Name', 'get_NonstdCount', 'get_ShapeType', 'get_Special', 'get_Type', 'set_MaterialName', 'set_Name', 'set_ShapeType', 'set_Type']
0.001666666666666667
['CompareTo', 'Eq